In [1]:
from typing import List, Tuple

import pandas as pd
import torch
import numpy as np

from module import Word2IdConverter, sentence2words

In [2]:
torch.cuda.is_available()

True

In [3]:
train_df = pd.read_csv('data/train.txt', sep='\t')
train_df.shape

(10672, 8)

In [4]:
converter = Word2IdConverter('data/mapping.csv')
n_words = converter.get_n_words()

In [5]:
results = map(sentence2words, train_df.title)
results = map(converter.word2id, results)
X = list(map(lambda x: torch.Tensor(x).long(), results))
len(X)

10672

In [6]:
y = torch.Tensor(train_df.category.map({'b': 0, 't': 1, 'e': 2, 'm': 3}).to_list()).long()

In [59]:
class CNN(torch.nn.Module):
    def __init__(self,
                 vocab_size: int,
                 input_size: int,
                 hidden_size: int,
                 output_size: int,
                 kernel_size: int):
        super(CNN, self).__init__()
        self._hidden_size = hidden_size
        self._kernel_size = kernel_size
        self.embedding = torch.nn.Embedding(vocab_size, input_size)
        self.conv = torch.nn.Conv2d(
            in_channels=1,
            out_channels=hidden_size,
            kernel_size=kernel_size,
            stride=1,
            padding=0,
            groups=1,
            bias=True
        )
        self.linear = torch.nn.Linear(hidden_size*5*99, output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x)
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = torch.max_pool2d(x, kernel_size=(self._kernel_size, self._kernel_size))
        x = x.view(-1, self._hidden_size*5*99)
        x = self.linear(x)
        return torch.log_softmax(x, dim=1)

In [75]:
dw = 300
dh = 50
n_class = 4
kernel_size = 3
cnn = CNN(vocab_size=n_words, input_size=dw, hidden_size=dh, output_size=n_class, kernel_size=kernel_size).cuda()

In [76]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

In [77]:
n_train_size = 9600
X_train = X[:n_train_size]
y_train = y[:n_train_size]
X_pad = torch.nn.utils.rnn.pad_sequence(X_train, batch_first=True)

batch_size = 128
n_epochs = 30
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    for i in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        y_pred = cnn(X_pad[i:i+batch_size].cuda())
        loss = criterion(y_pred, y_train[i:i+batch_size].cuda())
        loss.backward()
        optimizer.step()
        correct += (y_pred.argmax(1) == y_train[i:i+batch_size].cuda()).sum().item()
        running_loss += loss.item()
    
    print(f'epoch {epoch}- loss: {running_loss}, accuracy: {correct/y_train.shape[0]}')
print('DONE')

epoch 1- loss: 173.71395802497864, accuracy: 0.4721875
epoch 2- loss: 80.6879352927208, accuracy: 0.6117708333333334
epoch 3- loss: 62.52748650312424, accuracy: 0.7184375
epoch 4- loss: 47.35523107647896, accuracy: 0.7810416666666666
epoch 5- loss: 33.319269359111786, accuracy: 0.8380208333333333
epoch 6- loss: 24.347454965114594, accuracy: 0.890625
epoch 7- loss: 20.82871699333191, accuracy: 0.9016666666666666
epoch 8- loss: 16.88555385172367, accuracy: 0.9207291666666667
epoch 9- loss: 13.701288122683764, accuracy: 0.9388541666666667
epoch 10- loss: 8.890491854399443, accuracy: 0.9636458333333333
epoch 11- loss: 6.465276751667261, accuracy: 0.97375
epoch 12- loss: 4.327330857515335, accuracy: 0.98625
epoch 13- loss: 3.6924655716866255, accuracy: 0.9895833333333334
epoch 14- loss: 3.0161969298496842, accuracy: 0.9928125
epoch 15- loss: 2.5247365729883313, accuracy: 0.9951041666666667
epoch 16- loss: 2.6038650376722217, accuracy: 0.9942708333333333
epoch 17- loss: 2.82177910534665, acc

In [ ]:
y_pred.argmax(dim=1)[:10]